In [143]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [144]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [145]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /home/denys_herasymuk/UCU/4course_2term/Bachelor_Thesis/Code/fairness-variance


## Import dependencies

In [146]:
import os
import pandas as pd
from datetime import datetime, timezone

from configs.models_config_for_tuning import MODELS_CONFIG
from source.custom_initializers import create_config_obj, create_models_config_from_tuned_params_df
from source.custom_classes.data_loaders import CompasWithoutSensitiveAttrsDataset
from source.metrics_computation_interfaces import compute_metrics_multiple_runs

## Configs

In [147]:
config = create_config_obj(config_yaml_path=os.path.join('configs', 'experiment1_compas_config.yaml'))

TUNED_PARAMS_FILE_PATH = os.path.join('results', 'models_tuning', f'tuning_results_{config.dataset_name}_20230117__012024.csv')
SAVE_RESULTS_DIR_PATH = os.path.join('results', 'hypothesis_space',
                                     f'{config.dataset_name}_Metrics_{datetime.now(timezone.utc).strftime("%Y%m%d__%H%M%S")}')

## Models tuned hyper-parameters

In [148]:
models_tuned_params_df = pd.read_csv(TUNED_PARAMS_FILE_PATH)
models_tuned_params_df

,Dataset_Name,Model_Name,F1_Score,Accuracy_Score,Model_Best_Params
0,COMPAS_Without_Sensitive_Attributes,LogisticRegression,0.6785,0.6837,"{'C': 1, 'max_iter': 50, 'penalty': 'l2', 'sol..."
1,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,0.6952,0.6989,"{'criterion': 'gini', 'max_depth': 20, 'max_fe..."
2,COMPAS_Without_Sensitive_Attributes,RandomForestClassifier,0.6962,0.7008,"{'max_depth': 4, 'max_features': 0.6, 'min_sam..."
3,COMPAS_Without_Sensitive_Attributes,XGBClassifier,0.6971,0.7008,"{'lambda': 100, 'learning_rate': 0.1, 'max_dep..."
4,COMPAS_Without_Sensitive_Attributes,KNeighborsClassifier,0.7056,0.7112,"{'metric': 'minkowski', 'n_neighbors': 25, 'we..."
5,COMPAS_Without_Sensitive_Attributes,MLPClassifier,0.6834,0.6875,"{'activation': 'relu', 'hidden_layer_sizes': (..."


In [149]:
experiment_models_config = create_models_config_from_tuned_params_df(
    models_config_for_tuning=MODELS_CONFIG,
    models_tuned_params_df=models_tuned_params_df,
)

In [150]:
experiment_models_config.keys()

dict_keys(['DecisionTreeClassifier', 'LogisticRegression'])

## Load dataset

In [151]:
dataset = CompasWithoutSensitiveAttrsDataset(dataset_path='data/COMPAS.csv')
dataset.X_data.head()

,juv_fel_count,juv_misd_count,juv_other_count,priors_count,age_cat_25 - 45,age_cat_Greater than 45,age_cat_Less than 25,c_charge_degree_F,c_charge_degree_M
0,0.0,-2.340451,1.0,-15.010999,1,0,0,0,1
1,0.0,0.000000,0.0,0.000000,1,0,0,1,0
2,0.0,0.000000,0.0,0.000000,0,0,1,1,0
3,0.0,0.000000,0.0,6.000000,1,0,0,0,1
4,0.0,0.000000,0.0,7.513697,1,0,0,1,0


## Run experiments

In [152]:
multiple_runs_metrics_dct = compute_metrics_multiple_runs(dataset, config, experiment_models_config, SAVE_RESULTS_DIR_PATH, debug_mode=False)

Analyze models in one run:   0%|          | 0/2 [00:00<?, ?it/s]

##############################  [Model 1 / 2] Analyze DecisionTreeClassifier  ##############################
Model random_state:  101
Baseline X_train shape:  (4222, 9)
Baseline X_test shape:  (1056, 9)




2023-01-26 01:17:04 abstract_overall_variance_analyzer.py INFO    : Start classifiers testing by bootstrap


Classifiers testing by bootstrap: 100%|██████████| 100/100 [00:00<00:00, 183.67it/s]


2023-01-26 01:17:04 abstract_overall_variance_analyzer.py INFO    : Successfully tested classifiers by bootstrap
2023-01-26 01:17:07 abstract_overall_variance_analyzer.py INFO    : Successfully computed predict proba metrics

Analyze models in one run:  50%|█████     | 1/2 [00:09<00:09,  9.68s/it]





##############################  [Model 2 / 2] Analyze LogisticRegression  ##############################
Model random_state:  102
Baseline X_train shape:  (4222, 9)
Baseline X_test shape:  (1056, 9)




...

2023-01-26 01:17:28 abstract_overall_variance_analyzer.py INFO    : Start classifiers testing by bootstrap


Classifiers testing by bootstrap: 100%|██████████| 100/100 [00:00<00:00, 216.79it/s]


2023-01-26 01:17:29 abstract_overall_variance_analyzer.py INFO    : Successfully tested classifiers by bootstrap
2023-01-26 01:17:31 abstract_overall_variance_analyzer.py INFO    : Successfully computed predict proba metrics

Analyze models in one run:  50%|█████     | 1/2 [00:09<00:09,  9.74s/it]





##############################  [Model 2 / 2] Analyze LogisticRegression  ##############################
Model random_state:  202
Baseline X_train shape:  (4222, 9)
Baseline X_test shape:  (1056, 9)




2023-01-26 01:17:38 abstract_overall_variance_analyzer.py INFO    : Start classifiers testing by bootstrap


Classifiers testing by bootstrap: 100%|██████████| 100/100 [00:03<00:00, 31.70it/s]


2023-01-26 01:17:41 abstract_overall_variance_analyzer.py INFO    : Successfully tested classifiers by bootstrap
2023-01-26 01:17:44 abstract_overall_variance_analyzer.py INFO    : Successfully computed predict proba metrics

Multiple runs progress: 100%|██████████| 2/2 [00:46<00:00, 23.47s/it]